In [25]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import sys
import os
import time
import copy

In [26]:
class UAV(object):
    """
    docstring
    """
    UAV_set = {}
    beta = 100000 # 无人机逾期惩罚
    alpha = 1 # 无人机飞行成本
    def __init__(self,ID:int,free_time:float,next:int) -> None:
        self.ID = ID
        # self.now = now
        self.free_time = free_time
        self.next = next
        self.route = [(next,0)] # 记录无人机的路径与此次飞行成本
        UAV.UAV_set[self.ID] = self

    
    def __str__(self) -> str:
        return "ID:{} free_time:{} next:{}".format(self.ID,self.free_time,self.next)
    
    def print_route(self):
        print(self.route)
    
    def go_to_next(self,next:int,cost_time,end_time):
        all_cost = UAV.alpha * cost_time
        self.next = next
        self.free_time = self.free_time + cost_time
        if self.free_time > end_time:
            # all_cost += (self.free_time - end_time) * UAV.beta
            all_cost += UAV.beta
        self.route.append((next,all_cost))
    
    # 定义对比函数，以free_time为准
    def __lt__(self,other):
        return self.free_time < other.free_time
    
    @classmethod
    def get_UAV(cls):
        return cls.UAV_set
    
    @classmethod
    def get_UAV_num(cls):
        return len(cls.UAV_set)
    
    @classmethod
    def get_min_freetime(cls):
        # 寻找所有无人机中最小的free_time
        min_free_time = sys.maxsize
        min_free_time_UAV = None
        for uav_id in cls.UAV_set.keys():
            if cls.UAV_set[uav_id].free_time < min_free_time:
                min_free_time_UAV = uav_id
                min_free_time = cls.UAV_set[uav_id].free_time
        return min_free_time_UAV,min_free_time
    
    @classmethod
    def select_best_UAV(cls,current_time,target:int,time_matrix,end_time):
        cost={}
        for uav_id in cls.UAV_set.keys():
            temp=copy.deepcopy(cls.UAV_set[uav_id])
            temp.go_to_next(target,time_matrix[temp.next][target],end_time)
            cost[uav_id]=temp.route[-1][1]

        best_UAV = min(cost,key=cost.get)
        return best_UAV,cost[best_UAV]

In [27]:
# 读取数据txt文件
import numpy as np
import math


def read_graph(file_path):
    with open(file_path,'r') as f:
        data = f.readlines()
    count=int(data[0])
    data = [i.split('\n')[0] for i in data[1:]]
    data = [list(map(int,i.split())) for i in data]
    # 创建以data第一列为key的字典
    graph = {}
    for i in data:
        if i[0] not in graph.keys():
            graph[i[0]] = (i[1],i[2])
        else:
            graph[i[0]].append((i[1],i[2]))

    return count,graph

N,graph=read_graph(r'D:\WorkPath\VRPTW\1\graph.txt')

# 计算点之间的距离
def cal_distance(graph):
    distance = np.zeros((N,N))
    for i in range(0,N):
        for j in range(0,N):
            distance[i][j] = math.sqrt((graph[i][0]-graph[j][0])**2 + (graph[i][1]-graph[j][1])**2)
    return distance

distance = cal_distance(graph)

# 读取每个点的初始位置
def read_init_pos(file_path):
    with open(file_path,'r') as f:
        data = f.readlines()
    count=int(data[0])
    data = [i.split('\n')[0] for i in data[1:]]
    data = [list(map(int,i.split())) for i in data]
    init_pos = {}
    for i in data:
        init_pos[i[0]] = i[1]
    return count,init_pos
# distance
F,init_pos = read_init_pos(r'D:\WorkPath\VRPTW\1\initial_pos.txt')

# 读取任务信息
def read_task(file_path):
    with open(file_path,'r') as f:
        data = f.readlines()
    count=int(data[0])
    data = [i.split('\n')[0] for i in data[1:]]
    data = [list(map(int,i.split())) for i in data]
    task = {}
    for i in data:
        task[i[0]] = (i[1],i[2],i[3])
    return count,task

M,task = read_task(r'D:\WorkPath\DDPW\DPPW\1\road.txt')
# task

In [28]:
# 初始化无人机
for i in range(F):
    UAV(i,0,init_pos[i])
    
task_list=task.values()
# 将task_list按照最后一个数的大小排序
task_list=sorted(task_list,key=lambda x:x[2])

In [29]:
current_time=0
while task_list:
    start = task_list[0][0]
    end = task_list[0][1]
    end_time = task_list[0][2]
    start_time = end_time - distance[start][end]
    best_UAV,cost = UAV.select_best_UAV(current_time,start,distance,start_time)
    UAV.UAV_set[best_UAV].go_to_next(start,distance[UAV.UAV_set[best_UAV].next][start],start_time)
    UAV.UAV_set[best_UAV].go_to_next(end,distance[start][end],end_time)
    task_list.pop(0)
    current_time = UAV.get_min_freetime()[1]

In [33]:
print(UAV.get_UAV()[1].route)

[(1, 0), (1, 0.0), (4, 57.42821606144492), (4, 0.0), (9, 26.30589287593181)]
